In [ ]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S18'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

In [2]:
status="done"

In [3]:
print(status)

done


In [ ]:
# Cluster Robot Settings
muStart = 0.60
muEnd = 0.60
muIncr = 0.05
betaStart = 14
betaEnd = 14
betaIncr = 1
sigmaStart = 145
sigmaEnd = 145
sigmaIncr = 1
epsilonStart = 35
epsilonEnd = 35
epsilonIncr = 1
mu = muStart
while mu <= muEnd:
    beta = betaStart
    while beta <= betaEnd:
        sigma = sigmaStart
        while sigma <= sigmaEnd:
            epsilon = epsilonStart
            while epsilon <= epsilonEnd:
                suffixString =','+str(epsilon)+','+str(epsilonIterate)
                resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
                                                     epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
                seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
                seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
                seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
                seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
                outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
                resultsFile = open('RobotResults.txt','a+')
                resultsFile.write(outString)
                resultsFile.close()
                epsilon = epsilon + epsilonIncr
            sigma = sigma + sigmaIncr
        beta = beta + betaIncr
    mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [4]:
import numpy as np

Create a function of the parameters that returns the F score

In [198]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [185]:
entropy_F1(mu=0.5,muIncr=0.05,beta=33,sigma=449,epsilon=31.8,epsilonIterate=0)

S18-TokenReplace.txt
33
449
0.5
0.05
31.8
0
False
False
True

>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9984
NewMatrix Iterateblocks Total Time = 11.047514321282506


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9353
NewMatrix Iterateblocks Total Time = 11.405732677318156


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8135
NewMatrix Iterateblocks Total Time = 9.108074237592518


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 5238
NewMatrix Iterateblocks Total Time = 5.73466569185257


>>>>>>>>>>>>
Starting Iteration mu= 0.700000000000

0.72451483

Create functions for variable transformation.

In [199]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [200]:
inv_softplus(1)

0.541324854612918

In [201]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [202]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [220]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(50)), 
           'sigma_sp': (inv_softplus(40), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.7)),
           'muIncr_logit': (logit(0), logit(0.15)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [221]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    f1=entropy_F1(beta=beta_pos,
                  epsilon=epsilon_pos,
                  epsilonIterate=epsilonIterate_01,
                  muIncr=muIncr_01,
                  sigma=sigma_pos,
                  mu=mu_01)
    
    return logit(f1)

In [222]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9984
NewMatrix Iterateblocks Total Time = 12.010352770797908


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9353
NewMatrix Iterateblocks Total Time = 11.423873039893806


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8135
NewMatrix Iterateblocks Total Time = 8.974620432592928


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 5238
NewMatrix Iterateblocks Total Time = 5.652731854468584


>>>>>>>>>>>>
Starting Iteration mu= 0.7000000000000002
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!

0.9669684330973831

In [224]:
optimizer=None

In [225]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
)

In [226]:
optimizer.probe(
    params={"beta_sp": inv_softplus(33), 
            "sigma_sp": inv_softplus(449),
            "mu_logit":logit(0.5),
            "muIncr_logit":logit(0.05),
            "epsilon_sp":inv_softplus(31.8)},
    lazy=False,
)



>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9984
NewMatrix Iterateblocks Total Time = 11.200302894227207


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9353
NewMatrix Iterateblocks Total Time = 11.452218347229064


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8135
NewMatrix Iterateblocks Total Time = 9.211288628168404


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 5238
NewMatrix Iterateblocks Total Time = 5.699682458303869


>>>>>>>>>>>>
Starting Iteration mu= 0.7000000000000002
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!

In [227]:
optimizer.maximize(
    init_points=10,
    n_iter=20,
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------

>>>>>>>>>>>>
Starting Iteration mu= 0.5636935137423188
Starting Iteration epsilon= 504.3785407124369
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9619
NewMatrix Iterateblocks Total Time = 8.398568263277411

Precision = TP / (TP + FP) ......................... = 0.00873795
Recall = TP / (TP + FN) ............................ = 0.83744802
F-Measure .......................................... = 0.01729545
|  1        | -4.04     |  21.17    |  504.4    | -124.3    |  0.2562   |  136.9    |

>>>>>>>>>>>>
Starting Iteration mu= 0.5832623582581279
Starting Iteration epsilon= 130.82264553713838
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 5206
NewMatrix Iterateblocks Total Time = 2.4823880679905415

Precision = TP / (TP + FP) ......................... = 0.8


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.397733730264008


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.52988991793245


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.569862432777882


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 11.88412031903863


>>>>>>>>>>>>
Starting Ite


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.595163154415786


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.392744697630405


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.301163105294108


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 13.67517421580851


>>>>>>>>>>>>
Starting It


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.43308343924582


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.50514882709831


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.56307475361973


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.458594935946167


>>>>>>>>>>>>
Starting Iter


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 11.939994418993592


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 13.54096422251314


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.002449030056596


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.105297401547432


>>>>>>>>>>>>
Starting It

...


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 14.335882944054902


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 13.320226749405265


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 13.076948707923293


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.594239315018058


>>>>>>>>>>>>
Starting I


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.353114984929562


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.921250718645751


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 14.382489847950637


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.639956936240196


>>>>>>>>>>>>
Starting I


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.898585729300976


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.337585972622037


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.982010954990983


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.270297133363783


>>>>>>>>>>>>
Starting I


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.7377741150558


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.29202071391046


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.885651781223714


>>>>>>>>>>>>
Starting Iteration mu= 0.5359122915020872
Starting Iteration epsilon= 60.026236268102046
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8815
NewMatrix Iterateblocks Total Time = 12.676145631819963


>>>>>>>>>>>>
Starting Iter

RecursionError: maximum recursion depth exceeded in comparison

In [211]:
optimizer.res

[{'target': 0.9669684330973831,
  'params': {'beta_sp': 32.99999999999999,
   'epsilon_sp': 31.799999999999986,
   'muIncr_logit': -2.9444389791664403,
   'mu_logit': 0.0,
   'sigma_sp': 449.0}},
 {'target': -0.24790856220027935,
  'params': {'beta_sp': 21.16668071367561,
   'epsilon_sp': 504.3785407124369,
   'muIncr_logit': -124.325572097888,
   'mu_logit': 0.4191219406223893,
   'sigma_sp': 136.85888793929462}},
 {'target': -1.3208036989116143,
  'params': {'beta_sp': 5.108269416664427,
   'epsilon_sp': 130.82264553713838,
   'muIncr_logit': -81.97212416757868,
   'mu_logit': 0.5500365122017593,
   'sigma_sp': 395.61904444221557}},
 {'target': -0.1558555253545304,
  'params': {'beta_sp': 21.274130165213762,
   'epsilon_sp': 479.8240487859144,
   'muIncr_logit': -99.27272817695128,
   'mu_logit': 1.217329250469822,
   'sigma_sp': 58.07581151063127}},
 {'target': -5.653322838921686,
  'params': {'beta_sp': 33.701759636063024,
   'epsilon_sp': 292.4287890501312,
   'muIncr_logit': -55.

In [212]:
inv_logit(optimizer.max["target"])

0.72451483

In [213]:
params=optimizer.max["params"]

In [214]:
softplus(params["beta_sp"])

33.0

In [215]:
softplus(params["sigma_sp"])

449.0

In [216]:
inv_logit(params["mu_logit"])

0.5

In [217]:
inv_logit(params["muIncr_logit"])

0.05000000000000002

In [218]:
inv_logit(params["epsilonIterate_logit"])

KeyError: 'epsilonIterate_logit'

In [219]:
softplus(params["epsilon_sp"])

31.8